In [1]:
%matplotlib inline
from collections import defaultdict

import matplotlib.pylab as plt
from skimpy import skim
import numpy as np
import pandas as pd
from IPython.display import display, HTML
import seaborn as sns


In [2]:

import tensorflow as tf
import tensorflow_probability as tfp

from bayesianquilts.models.logistic_regression import LogisticRegression

2024-01-03 17:30:28.138664: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-03 17:30:28.165382: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-03 17:30:28.166038: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 17:30:28.673775: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
lr_model = LogisticRegression(dim_regressors=100, regression_interact=None)

# Load the dataset

Using the example from here: https://www.tensorflow.org/guide/core/logistic_regression_core

In [3]:
from ucimlrepo import fetch_ucirepo 


In [4]:
# fetch dataset 
cdc_diabetes_health_indicators = fetch_ucirepo(id=891) 
  
# data (as pandas dataframes) 
X = cdc_diabetes_health_indicators.data.features 
y = cdc_diabetes_health_indicators.data.targets 

In [5]:
X

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1,1,1,40,1,0,0,0,0,1,...,1,0,5,18,15,1,0,9,4,3
1,0,0,0,25,1,0,0,1,0,0,...,0,1,3,0,0,0,0,7,6,1
2,1,1,1,28,0,0,0,0,1,0,...,1,1,5,30,30,1,0,9,4,8
3,1,0,1,27,0,0,0,1,1,1,...,1,0,2,0,0,0,0,11,3,6
4,1,1,1,24,0,0,0,1,1,1,...,1,0,2,3,0,0,0,11,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,1,1,1,45,0,0,0,0,1,1,...,1,0,3,0,5,0,1,5,6,7
253676,1,1,1,18,0,0,0,0,0,0,...,1,0,4,0,0,1,0,11,2,4
253677,0,0,1,28,0,0,0,1,1,0,...,1,0,1,0,0,0,0,2,5,2
253678,1,0,1,23,0,0,0,0,1,1,...,1,0,3,0,0,0,1,7,5,1


In [7]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'

features = ['radius', 'texture', 'perimeter', 'area', 'smoothness', 'compactness',
            'concavity', 'concave_poinits', 'symmetry', 'fractal_dimension']
column_names = ['id', 'diagnosis']

for attr in ['mean', 'ste', 'largest']:
  for feature in features:
    column_names.append(feature + "_" + attr)

dataset = pd.read_csv(url, names=column_names)

skim(dataset)


╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 569    │ │ float64     │ 30    │                                                          │
│ │ Number of columns │ 32     │ │ int64       │ 1     │                                                          │
│ └───────────────────┴────────┘ │ string      │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name  ┃ NA ┃ NA % ┃ mean     ┃ sd        ┃ p0      ┃ p25    ┃ p50    ┃ p75     ┃ p100      ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩  │
│ │ id           │  0 │    0 │ 30000000 │ 130000000 │    8700 │ 870000 │ 910000 │ 8800000 │ 910000000 │   ▇    │  │
│ │ radius_mean  │  0 │    0 │       14 │       3.5 │       7 │     12 │     13 │      16 │        28 │  ▂▇▃▂  │  │
│ │ texture_mean │  0 │    0 │       19 │       4.3 │     9.7 │     16 │     19 │      22 │        39 │  ▂▇▆▂  │  │
│ │ perimeter_me │  0 │    0 │       92 │        24 │      44 │     75 │     86 │     100 │       190 │  ▂▇▃▂  │  │
│ │ an           │    │      │          │           │         │        │        │         │           │        │  │
│ │ area_mean    │  0 │    0 │      650 │       350 │     140 │    420 │    550 │     780 │      2500 │  ▇▆▂▁  │  │
│ │ smoothness_m │  0 │    0 │    0.096 │     0.014 │   0.053 │  0.086 │  0.096 │    0.11 │      0.16 │   ▅▇▃  │  │
│ │ ean          │    │      │          │           │         │        │        │         │           │        │  │
│ │ compactness_ │  0 │    0 │      0.1 │     0.053 │   0.019 │  0.065 │  0.093 │    0.13 │      0.35 │  ▇▇▃▁  │  │
│ │ mean         │    │      │          │           │         │        │        │         │           │        │  │
│ │ concavity_me │  0 │    0 │    0.089 │      0.08 │       0 │   0.03 │  0.062 │    0.13 │      0.43 │  ▇▃▂▁  │  │
│ │ an           │    │      │          │           │         │        │        │         │           │        │  │
│ │ concave_poin │  0 │    0 │    0.049 │     0.039 │       0 │   0.02 │  0.034 │   0.074 │       0.2 │  ▇▃▃▁  │  │
│ │ its_mean     │    │      │          │           │         │        │        │         │           │        │  │
│ │ symmetry_mea │  0 │    0 │     0.18 │     0.027 │    0.11 │   0.16 │   0.18 │     0.2 │       0.3 │ ▁▇▇▂▁  │  │
│ │ n            │    │      │          │           │         │        │        │         │           │        │  │
│ │ fractal_dime │  0 │    0 │    0.063 │    0.0071 │    0.05 │  0.058 │  0.062 │   0.066 │     0.097 │  ▅▇▃▁  │  │
│ │ nsion_mean   │    │      │          │           │         │        │        │         │           │        │  │
│ │ radius_ste   │  0 │    0 │     0.41 │      0.28 │    0.11 │   0.23 │   0.32 │    0.48 │       2.9 │   ▇▁   │  │
│ │ texture_ste  │  0 │    0 │      1.2 │      0.55 │    0.36 │   0.83 │    1.1 │     1.5 │       4.9 │  ▇▆▁   │  │
│ │ perimeter_st │  0 │    0 │      2.9 │         2 │    0.76 │    1.6 │    2.3 │     3.4 │        22 │   ▇▁   │  │
│ │ e            │    │      │          │           │   

In [11]:
X.columns

Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')